In [ ]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
from src.Map import Map
from src.Tabu import Tabu
from src.TwoStage import TwoStage
from src.L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl
from gurobipy import *
import csv

**Modelling Parameters**

In [ ]:
rides = 3
bus = 2
scenarios = 10
MIPGap = 0.001
TimeLimit = 18000
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [ ]:
mappy = Map(rides, seed=200)

**Plotting Functions**

In [ ]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


#### Model formulation

In [ ]:
def Mod1(aux=False): # Standard DARP model
    t1 = time.time()
    drpstw = mp(mappy,bus=bus,scenarios=scenarios, probability=probability)
    drpstw.initialize()
    drpstw.setMIP()
    drpstw.variables.th.obj=0
    if aux:
        drpstw.setLowerBound()
        for s in range(drpstw.scenarios):
            drpstw.submodel[s].model.params.MIPGap = MIPGap
#             drpstw.submodel[s].model.params.OutputFlag = 1
    else:
        drpstw.model.params.TimeLimit = TimeLimit
        drpstw.model.optimize()
    t1 = time.time() - t1
    return drpstw, t1

In [ ]:
def Mod2(model=None): # Standard Two-Stage Optimization for DARP problem
    t2 = time.time()
    twosp = TwoStage(mappy, bus=bus, scenarios=scenarios, probability=probability)
    twosp.model.params.TimeLimit = TimeLimit
    if model is not None:
        for i,j,k in twosp.MP.variables.x.keys():
            twosp.MP.variables.x[i,j,k].start = model.variables.x[i,j,k].X
    twosp.optimize()
    
    master =sum(twosp.MP.variables.x[i, j, k].X * twosp.parameters.distance[i, j]
                     + twosp.MP.variables.h[i, j].X
                     # * twosp.MP.variables.w[i]
                     for i, j in twosp.parameters.edges for k in range(twosp.MP.bus)) + \
            sum(twosp.MP.variables.p_l[i].X for i in twosp.parameters.pickup)
    
    savings = (sum((1 / twosp.scenarios) * (twosp.parameters.distance[i, j] * (twosp.variables.xs[i, j, k, s].X -
                                                                               twosp.MP.variables.x[i, j, k].X)
                                             + (twosp.variables.hs[i, j, s].X - twosp.MP.variables.h[i, j].X))
                                             for i, j in twosp.parameters.edges
                                             for k in range(twosp.MP.bus) for s in twosp.parameters.xi) +
                                    sum((1 / twosp.scenarios) * (twosp.variables.p_ls[i, s].X - twosp.MP.variables.p_l[i].X)
                                             for i in twosp.parameters.pickup for s in twosp.parameters.xi))
    
    t2 = time.time() - t2
    return twosp, savings,master, t2

In [ ]:
def Mod3(): # L-Shaped method implementation for DARP with Initialisation
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.model.params.MIPGap = MIPGap
#     if lshaped.model.runtime < TimeLimit:
#         lshaped.model.params.MIPGap
    lshaped.optimize(skip_init=False)
    savings = sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())
    t3 = time.time() - t3
    return lshaped, savings, t3

In [ ]:
def Mod4(lshaped): # Tabu search heuristic for DARP
    t4 = time.time()
    if lshaped.submodel is not None:
        tabu = Tabu(lshaped,tabu_iterations=800, tabu_status=rides + 2, rtoptm=5, subset=20, tsp=False)
        tabu.tsp = True
    else:
        tabu = Tabu(lshaped,tabu_iterations=800, tabu_status=rides + 2, rtoptm=5, subset=20, tsp=True)
    tabu.tabuheuristic()
    t4 = time.time() - t4
    return tabu, t4

In [ ]:
def Mod5(): # L-Shaped method implementation for DARP without Initialisation
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.optimize(skip_init=True)
    savings = sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())
    t3 = time.time() - t3
    return lshaped, savings, t3

### Test Block

In [ ]:
TimeLimit = 18000
tests = [(3,2),(4,2),(5,2),(6,2),(7,2),(9,3),(10,3),(11,3),(13,4),(15,4),(18,5),(20,5),(22,6),(24,6),(25,7),(27,8)]
scenarioslist = [10,20,30]

csv_file = open('./Reports/Report.csv', 'w', newline='')
write = csv.writer(csv_file, delimiter=',')
write.writerow(('Rides','Bus','Scenarios', 'DRPObj', 'DRPTime', 'TSPObj', 'TSPSave','TSPGap','TSPTime','LshapeNsObj','LshapeNsSave','LshapeNsGap','LshapeNsTime','LshapeObj','LshapeSave','LshapeGap','LshapeTime','TabuObj','TabuSave','TabuTime'))
csv_file.close()
for s in scenarioslist:
    scenarios = s
    for test in tests:
        
        rides, bus = test
        mappy = Map(rides, seed=200)

        #Darp-stw
        try:
            drpstw, t1 = Mod1()
            if drpstw.model.status in [2,9]:
                drpobj = drpstw.model.ObjVal
                drptime = t1
            else:
                drpobj = 'NA'
                drptime = t1
        except (GurobiError, AttributeError, KeyboardInterrupt):
            drpobj = 'Failed'
            drptime = 'Failed'
        #Two-Stage
        try:    
            twostage, savings, master, t1 = Mod2()
            if twostage.model.status in [2,9]:
                twostageobj = twostage.model.ObjVal
                twostagesave = savings
                twostagegap = twostage.model.MIPGap
                twostagetime = t1
            else:
                twostageobj = 'NA'
                twostagesave = 'NA'
                twostagegap = 'NA'
                twostagetime = t1
        except (GurobiError, AttributeError, KeyboardInterrupt):
            twostageobj = 'Failed'
            twostagesave = 'Failed'
            twostagegap = 'Failed'
            twostagetime = 'Failed'
        #L-shaped no initialisation
        try:    
            lshaped, apprxskip, t2 = Mod5()
            if lshaped.model.status in [2,9]:
                lshapedobjns = lshaped.model.ObjVal
                lshapedsavens = lshaped.variables.th.X
                lshapedgapns = lshaped.model.MIPGap
                lshapedtimens = t2
            else:
                lshapedobjns = 'NA'
                lshapedgapns = 'NA'
                lshapedtimens = t2
                lshapedsavens = 'NA'
        except (GurobiError, AttributeError, KeyboardInterrupt):
            lshapedobjns = 'Failed'
            lshapedsavens = 'Failed'
            lshapedgapns = 'Failed'
            lshapedtimens = 'Failed'
        #L-shaped
        try:    
            lshaped, apprx, t2 = Mod3()
            if lshaped.model.status in [2,9]:
                lshapedobj = lshaped.model.ObjVal
                lshapedsave = lshaped.variables.th.X
                lshapedgap = lshaped.model.MIPGap
                lshapedtime = t2
            else:
                lshapedobj = 'NA'
                lshapedgap = 'NA'
                lshapedtime = t2
                lshapedsave = 'NA'
        except (GurobiError, AttributeError, KeyboardInterrupt):
            lshapedobj = 'Failed'
            lshapedsave = 'Failed'
            lshapedgap = 'Failed'
            lshapedtime = 'Failed'
        #Tabu
        try:    
            tabu, t3 = Mod4(lshaped)
            tabuobj = tabu.best[-1]
            tabusave = tabu.best[-3]
            tabutime = t3
        except (GurobiError, AttributeError, KeyboardInterrupt):
            tabuobj = 'Failed'
            tabusave = 'Failed'
            tabutime = 'Failed'

        csv_file = open('./Reports/Report.csv', 'a', newline='')
        write = csv.writer(csv_file, delimiter=',')
        write.writerow((str(rides),
                        str(bus),
                        str(scenarios),
                        str(drpobj),
                        str(drptime),
                        str(twostageobj),
                        str(twostagesave),
                        str(twostagegap),
                        str(twostagetime),
                        str(lshapedobjns),
                        str(lshapedsavens),
                        str(lshapedgapns),
                        str(lshapedtimens),
                        str(lshapedobj),
                        str(lshapedsave),
                        str(lshapedgap),
                        str(lshapedtime),
                        str(tabuobj),
                        str(tabusave),
                        str(tabutime)))
        csv_file.close()
        

### DARP-STW Model

#### Model Solution 

In [ ]:
drpstw, t1 = Mod1()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(drpstw.bus)}
for i, j in drpstw.parameters.edges:
    for k in range(drpstw.bus):
        if drpstw.variables.x[i,j,k].X > 0.5:
            if j != drpstw.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objdrpstw = drpstw.model.ObjVal
displaygraph(nodes, edges,'DARP-STW')
nodes,edges

### Two-Stage Stochastic Model

#### Model Solution

In [ ]:
twosp, savings, master, t2 = Mod2()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(twosp.bus)}
for i, j in twosp.parameters.edges:
    for k in range(twosp.bus):
        if twosp.MP.variables.x[i,j,k].X > 0.5:
            if j != twosp.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objtwosp = twosp.model.ObjVal
displaygraph(nodes, edges,'Two-Stage')
nodes,edges

### L-Shaped Method

#### Model Solution

In [ ]:
lshaped, apprx, t3 = Mod3()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

In [ ]:
plot_trend(lshaped.upperbounds[2:],lshaped.lowerbounds[2:],0)

In [ ]:
mostcancel = {i: 0 for i in lshaped.parameters.pickup}
for i in lshaped.parameters.pickup:
    for s in range(lshaped.scenarios):
        mostcancel[i] += 2 - lshaped.submodel[s].sim.alpha[i] - lshaped.submodel[s].sim.alpha[i+lshaped.parameters.rides]
mostcancel = sorted(mostcancel.items(), key=lambda x: (x[1]))
mostcancel

In [ ]:
lshaped.printsol()

In [ ]:
from _datetime import timedelta
str(timedelta(minutes=tabu.best[0][2].time))

In [ ]:
sum((1/scenarios)*lshaped.submodel[i].model.ObjVal for i in lshaped.submodel.keys())

### Tabu-Heuristic

#### Model Solution

In [ ]:
tabu, t4 = Mod4(lshaped)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabu.bus:
    for i in tabu.best[k]:
        n = tabu.best[k][i]
        if i != tabu.model.last:
            edges[k].append((i,n.next.key))
objtabu = tabu.best[-2]
displaygraph(nodes, edges,'Tabu')    
nodes,edges, tabu.best

In [ ]:
# tabu.best[0] = LinkedList()
# [0,1,4,2,6,7,9,11]
for i in tabu.bestcandidate[0]:
    print(tabu.bestcandidate[0][i].bus)

### L-Shaped Method (without Initialisation)

#### Model Solution

In [ ]:
lshaped_skp, t5 = Mod5()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabutsp.bus:
    for i in tabutsp.best[k]:
        n = tabutsp.best[k][i]
        if i != tabutsp.model.last:
            edges[k].append((i,n.next.key))
objtabutsp = tabutsp.best[-2]
displaygraph(nodes, edges,'TabuTSP')    
nodes,edges, tabutsp.best

In [ ]:
t3,t4